In [16]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

# path to the model weights files.
weights_path = '/home/user/models/pretrained/vgg16_weights.h5'
top_model_weights_path = '/home/user/models/top_tuned/bottleneck_fc_model_2class.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/home/user/Документы/convnets/transfer-learning-keras/dataset/training'
validation_data_dir = '/home/user/Документы/convnets/transfer-learning-keras/dataset/validation'
evaluation_data_dir = '/home/user/Документы/convnets/transfer-learning-keras/dataset/evaluation'
nb_train_samples = 3000
nb_validation_samples = 1000
nb_evaluation_samples = 1000
epochs = 50
batch_size = 20

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, 
                               input_tensor=Input(shape=(224, 224, 3)))
print('VGG16 model loaded.')

VGG16 model loaded.


In [17]:
# build a classifier model to put on top of the convolutional model
top_model = base_model.output
top_model = Flatten(name='flatten')(top_model)
top_model = Dense(512, activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(1, activation='sigmoid')(top_model)

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model)

# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in base_model.layers:
    layer.trainable = False
for layer in model.layers:
    print("{}: {}".format(layer, layer.trainable))

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fad9c0f3150>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fad6765f910>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fad58c29050>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fad58ccd450>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fad58bf8990>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fad58b3db50>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fad58b51f50>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fad58b58950>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fad58aefe50>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fad58b035d0>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fad58b09b10>: False
<tensorflow.python.keras.layers.convolutional.Conv2D

In [20]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
)

test_datagen = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_datagen.mean = mean
test_datagen.mean = mean

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="rgb",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    color_mode="rgb",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    evaluation_data_dir,
    color_mode="rgb",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary')

print("[INFO] training head...")
H = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 3000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
[INFO] training head...
Epoch 1/50
150/150 [==============================] - 1329s 9s/step - loss: 1.0168 - accuracy: 0.9153 - val_loss: 0.4231 - val_accuracy: 0.9630
Epoch 2/50
150/150 [==============================] - 1334s 9s/step - loss: 0.9136 - accuracy: 0.9210 - val_loss: 0.4420 - val_accuracy: 0.9660
Epoch 3/50
 29/150 [====>.........................] - ETA: 16:18 - loss: 0.6545 - accuracy: 0.9466

KeyboardInterrupt: 

In [9]:
print("[INFO] evaluating after fine-tuning network head...")
test_generator.reset()
predIdxs = model.predict_generator(test_generator,
    steps=(nb_evaluation_samples // batch_size) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(test_generator.classes, predIdxs,
    target_names=test_generator.class_indices.keys()))
WARMUP_PLOT_PATH = os.path.sep.join(["output", "warmup.png"])
plot_training(H, 50, config.WARMUP_PLOT_PATH)

model.save_weights('/home/user/models/fine_tuned/fine_tuned_2class.h5')

[INFO] evaluating after fine-tuning network head...


NameError: name 'test_generator' is not defined